<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_teachability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatting with TeachableAgent

Conversational assistants based on LLMs can remember the current chat with the user, and can even demonstrate in-context learning of things that the user teaches the assistant during the chat. But these memories and learnings are lost once the chat is over, or when a single chat grows too long for the LLM to handle effectively. In subsequent chats, the user is forced to repeat any necessary instructions over and over.

`TeachableAgent` addresses these limitations by persisting user teachings across chat boundaries in long-term memory (a vector database). Memory is saved to disk at the end of each chat, then loaded from disk at the start of the next. Instead of copying all of memory into the context window, which would eat up valuable space, individual memories (called memos) are retrieved into context as needed. This allows the user to teach frequently used facts and skills to the teachable agent just once, and have it remember them in later chats.

In making decisions about memo storage and retrieval, `TeachableAgent` calls an instance of `TextAnalyzerAgent` to analyze pieces of text in several different ways. This adds extra LLM calls involving a relatively small number of tokens. These calls can add a few seconds to the time a user waits for a response.

## Construct Agents
For this walkthrough, we start by resetting the teachable agent's memory store. This deletes any memories from prior conversations that may be stored on disk.

In [1]:
%cd ..

from autogen.agentchat.contrib.teachable_agent import TeachableAgent
from autogen import ChatCompletion
from townhall.agents.user_agent import UserAgent
from townhall.services.chat_service import ChatService
from settings import CONFIG_LIST

llm_config = {
    "request_timeout": 60,
    "config_list": CONFIG_LIST,
    "use_cache": True,  # Use False to explore LLM non-determinism.
}

teach_config={
    "verbosity": 0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
    "reset_db": True,  # Set to True to start over with an empty database.
    "path_to_db_dir": "./tmp/notebook/teachable_agent_db",  # Path to the directory where the database will be stored.
    "recall_threshold": 1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
}

try:
    from termcolor import colored
except ImportError:
    def colored(x, *args, **kwargs):
        return x

ChatCompletion.start_logging()

teachable_agent = TeachableAgent(
    name="teachableagent",
    llm_config=llm_config,
    teach_config=teach_config)

CHAT_SERVICE = ChatService(
    config_list=CONFIG_LIST,
    assistants=[teachable_agent],
    user_proxy=UserAgent()
)


/mnt/e/workspace/townhall

CLEARING MEMORY


## Learning new facts
Let's teach the agent some facts it doesn't already know, since they are more recent than GPT-4's training data.

In [2]:
text = "What is the Vicuna model?"
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)

User (to teachableagent):

What is the Vicuna model?

--------------------------------------------------------------------------------
What is the Vicuna model?

--------------------------------------------------------------------------------
teachableagent (to User):

I'm sorry, but I couldn't find any information on a specific "Vicuna model." It's possible that the term may refer to something specific within a certain context or industry. Could you please provide more details or clarify your question?

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
User (to teachableagent):



--------------------------------------------------------------------------------
teachableagent (to User):

I'm sorry, but I still don't have any information regarding a "Vicuna model." It's possible that it may be a term or concept specific to a certain field or industry that is unfamiliar to me. If you can provi

In [3]:
text = "Vicuna is a 13B-parameter language model released by Meta."
CHAT_SERVICE.initiate_chat(message=text, clear_history=False)

User (to teachableagent):

Vicuna is a 13B-parameter language model released by Meta.

--------------------------------------------------------------------------------
Vicuna is a 13B-parameter language model released by Meta.

--------------------------------------------------------------------------------
teachableagent (to User):

Ah, I see. My apologies for the confusion earlier. Yes, you are correct. Vicuna is a language model released by Meta (formerly known as Facebook). It is based on the GPT-3 architecture and has been trained on a vast amount of text data to generate human-like responses. With 13 billion parameters, Vicuna is designed to understand and generate coherent text across various topics and contexts. If you have any more questions or need further information, feel free to ask!

--------------------------------------------------------------------------------


In [4]:
text = "What is the Orca model?"
CHAT_SERVICE.initiate_chat(message=text, clear_history=False)

User (to teachableagent):

What is the Orca model?

--------------------------------------------------------------------------------
What is the Orca model?

--------------------------------------------------------------------------------
teachableagent (to User):

I apologize for any confusion, but as of my current knowledge, there is no widely recognized or popular "Orca model" that I am aware of. It's possible that the term may refer to a specific model or concept within a certain context or industry. Can you please provide more details or clarify your question so that I can better understand and assist you?

--------------------------------------------------------------------------------


In [5]:
text = "Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks."
CHAT_SERVICE.initiate_chat(message=text, clear_history=False)

User (to teachableagent):

Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks.

--------------------------------------------------------------------------------
Orca is a 13B-parameter language model released by Microsoft. It outperforms Vicuna on most tasks.

--------------------------------------------------------------------------------
teachableagent (to User):

Thank you for providing the additional information. However, as an AI language model, my responses are based on pre-existing knowledge, and I am not always aware of the most recent developments or releases. If the Orca model is indeed a 13B-parameter language model released by Microsoft and outperforms Vicuna on various tasks, then it sounds like an interesting and promising advancement in the field of natural language processing. I recommend referring to official sources or recent publications from Microsoft for more accurate and up-to-date information on the Orca model and it

Let's end our first chat here. The following function needs to be called at the end of each chat, so that `TeachableAgent` can store what the user has taught it.

In [6]:
teachable_agent.learn_from_user_feedback()


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER


/home/james/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:23<00:00, 3.51MiB/s]


Now let's start a new chat by clearing the previous chat's history. At this point, common LLM-based assistants would forget everything from the last chat. But `TeachableAgent` can retrieve memories from its vector DB as needed, allowing it to recall and reason over facts that the user taught it in earlier conversations.

In [7]:
text = "How does the Vicuna model compare to the Orca model?"
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)

User (to teachableagent):

How does the Vicuna model compare to the Orca model?

--------------------------------------------------------------------------------
How does the Vicuna model compare to the Orca model?

--------------------------------------------------------------------------------
teachableagent (to User):

The Vicuna model and the Orca model are both 13B-parameter language models released by different companies. However, the Orca model has been found to outperform the Vicuna model on most tasks.

--------------------------------------------------------------------------------


## Learning user preferences
Now let's teach the agent some of our preferences. Suppose that we frequently post short summaries of new papers for our team to read, and we want the teachable agent to help us do this faster.

In [8]:
text = """Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)

User (to teachableagent):

Please summarize this abstract.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in

But that's unstructured. So let's teach the agent our preference.

In [9]:
text = """Please summarize this abstract.
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a generic infrastructure to build diverse applications of various complexities and LLM capacities. Empirical studies demonstrate the effectiveness of the framework in many example applications, with domains ranging from mathematics, coding, question answering, operations research, online decision-making, entertainment, etc.
"""
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)

User (to teachableagent):

Please summarize this abstract.
When I'm summarizing an abstract, I try to make the summary contain just three short bullet points:  the title, the innovation, and the key empirical results.

AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation
Qingyun Wu, Gagan Bansal, Jieyu Zhang, Yiran Wu, Beibin Li, Erkang Zhu, Li Jiang, Xiaoyun Zhang, Shaokun Zhang, Jiale Liu, Ahmed Hassan Awadallah, Ryen W White, Doug Burger, Chi Wang
AutoGen is an open-source framework that allows developers to build LLM applications via multiple agents that can converse with each other to accomplish tasks. AutoGen agents are customizable, conversable, and can operate in various modes that employ combinations of LLMs, human inputs, and tools. Using AutoGen, developers can also flexibly define agent interaction behaviors. Both natural language and computer code can be used to program flexible conversation patterns for different applications. AutoGen serves as a gener

That's much better, but will the teachable agent remember these preferences in the future, for a different paper? Let's start a new chat to find out!

In [10]:
teachable_agent.learn_from_user_feedback()

text = """Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit more general intelligence than previous AI models. We discuss the rising capabilities and implications of these models. We demonstrate that, beyond its mastery of language, GPT-4 can solve novel and difficult tasks that span mathematics, coding, vision, medicine, law, psychology and more, without needing any special prompting. Moreover, in all of these tasks, GPT-4's performance is strikingly close to human-level performance, and often vastly surpasses prior models such as ChatGPT. Given the breadth and depth of GPT-4's capabilities, we believe that it could reasonably be viewed as an early (yet still incomplete) version of an artificial general intelligence (AGI) system. In our exploration of GPT-4, we put special emphasis on discovering its limitations, and we discuss the challenges ahead for advancing towards deeper and more comprehensive versions of AGI, including the possible need for pursuing a new paradigm that moves beyond next-word prediction. We conclude with reflections on societal influences of the recent technological leap and future research directions."""
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER
User (to teachableagent):

Please summarize this abstract.

Sparks of Artificial General Intelligence: Early experiments with GPT-4
Sébastien Bubeck, Varun Chandrasekaran, Ronen Eldan, Johannes Gehrke, Eric Horvitz, Ece Kamar, Peter Lee, Yin Tat Lee, Yuanzhi Li, Scott Lundberg, Harsha Nori, Hamid Palangi, Marco Tulio Ribeiro, Yi Zhang
Artificial intelligence (AI) researchers have been developing and refining large language models (LLMs) that exhibit remarkable capabilities across a variety of domains and tasks, challenging our understanding of learning and cognition. The latest model developed by OpenAI, GPT-4, was trained using an unprecedented scale of compute and data. In this paper, we report on our investigation of an early version of GPT-4, when it was still in active development by OpenAI. We contend that (this early version of) GPT-4 is part of a new cohort of LLMs (along with ChatGPT and Google's PaLM for example) that exhibit mor

## Learning new skills
Finally, let's extend the teachable agent's capabilities by teaching it a new skill for accomplishing a challenging type of task. 

The [Sparks of AGI](https://arxiv.org/abs/2303.12712) paper evaluated GPT-4 on math problems like the following, which it could only solve 32% of the time.

In [11]:
text = """Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)

User (to teachableagent):

Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
teachableagent (to User):

To modify the left-hand side of the equation to make the right-hand side equal to 99, we can follow the following steps:

1. Since we need the result to be 99, let's start by swapping the prod

This time, lets teach the agent a reliable strategy for solving such tasks.

In [12]:
text = """Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".
"""
CHAT_SERVICE.initiate_chat(message=text, clear_history=False)

User (to teachableagent):

Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".

Here's some advice that may help:
1. Let E denote the original number on the right.
2. Let F denote the final number on the right.
3. Calculate the difference between the two, G = F - E.
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed.
5. Calculate J = G / H. This is the number of times that H divides into G.
6. Verify that J is an integer, and that H * J = G.
7. Find the number on the left which is multiplied by H, and call it K.
8. Change K to K + J.
9. Recompute the value on the left, and verify that it equals F.
Finally, write down your solution as: "The solution is: A * B + C * D".


--

When given this advice, GPT-4 can solve such problems over 95% of the time. But can the teachable agent remember the strategy so the user doesn't have to explain it over and over? As before, let's start a new chat to find out.

In [13]:
teachable_agent.learn_from_user_feedback()

text = """Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
CHAT_SERVICE.initiate_chat(message=text, clear_history=True)


REVIEWING CHAT FOR USER TEACHINGS TO REMEMBER
User (to teachableagent):

Consider the identity:
9 * 4 + 6 * 6 = 72
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 99?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
teachableagent (to User):

The solution is: 10 * 4 + 6 * 6

Let's go through the steps:

1. Let E denote the original number on the right: E = 72
2. Let F denote the final number on the right: F = 99
3. Calculate the difference between the two, G = F - E: G = 99 - 72 = 27
4. Examine the numbers on the left one by one until finding one that divides evenly into G, where negative integers are allowed: 27 can be divided evenly by 9.
5. Calculate J = G / H. This is the number of times that H divides into G: J = 27 / 9 = 3
6. Verify that J is an int

As a final check, let's test the teachable agent's newly learned skill on a separate instance of the task.

In [14]:
text = """Consider the identity:
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".
"""
CHAT_SERVICE.initiate_chat(message=text, clear_history=False)

User (to teachableagent):

Consider the identity:
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
Consider the identity:
8 * 3 + 7 * 9 = 87
Can you modify exactly one integer (and not more than that!) on the left hand side of the equation so the right hand side becomes 59?
-Let's think step-by-step, write down a plan, and then write down your solution as: "The solution is: A * B + C * D".


--------------------------------------------------------------------------------
teachableagent (to User):

The solution is: 8 * 3 + 6 * 9

Let's go through the steps:

1. Let E denote the original number on the right: E = 87
2. Let F denote the final number on the right: F = 80
3. Calculate the dif